In [1]:
import mne
from pathlib import Path
import numpy as np
from mnelab.io.writers import write_edf

In [17]:
# same function as mne but support nan values (const physical calues)
def write_edf_nan(fname, raw):
    """Export raw to EDF/BDF file (requires pyEDFlib)."""
    import pyedflib

    suffixes = Path(fname).suffixes
    ext = "".join(suffixes[-1:])
    if ext == ".edf":
        filetype = pyedflib.FILETYPE_EDFPLUS
        dmin, dmax = -32768, 32767
    elif ext == ".bdf":
        filetype = pyedflib.FILETYPE_BDFPLUS
        dmin, dmax = -8388608, 8388607
    data = raw.get_data() * 1e6  # convert to microvolts
    fs = raw.info["sfreq"]
    nchan = raw.info["nchan"]
    ch_names = raw.info["ch_names"]
    if raw.info["meas_date"] is not None:
        meas_date = raw.info["meas_date"]
    else:
        meas_date = None
    prefilter = (f"{raw.info['highpass']}Hz - "
                 f"{raw.info['lowpass']}")
    pmin, pmax = data.min(axis=1), data.max(axis=1)
    f = pyedflib.EdfWriter(fname, nchan, filetype)
    channel_info = []
    data_list = []
    for i in range(nchan):
        channel_info.append(dict(label=ch_names[i],
                                 dimension="uV",
                                 sample_rate=fs,
                                 physical_min=-5000,
                                 physical_max=5000,
                                 digital_min=dmin,
                                 digital_max=dmax,
                                 transducer="",
                                 prefilter=prefilter))
        data_list.append(data[i])
    f.setTechnician("Exported by MNELAB")
    f.setSignalHeaders(channel_info)
    if raw.info["meas_date"] is not None:
        f.setStartdatetime(meas_date)
    # note that currently, only blocks of whole seconds can be written
    f.writeSamples(data_list)
    for annot in raw.annotations:
        f.writeAnnotation(annot["onset"], annot["duration"], annot["description"])

In [1]:
import mne
subj = '44'
depth = ['RAH1', 'LAH1', 'RA1', 'LA1', 'LEC1', 'REC1', 'RPHG1', 'LPHG1', 'RMH1', 'LMH1', 'RAH2', 'LAH2', 'RA2', 'LA2', 'LEC2', 'REC2', 'RPHG2', 'LPHG2', 'RMH2', 'LMH2']
if subj in ['018', '025']:
    depth = [x for x in depth+['RA3', 'RA4'] if x not in ['RA1', 'RA2']]
raw = mne.io.read_raw_edf(f'C:\\UCLA\\P{subj}_full_filtered_mtl.edf')
raw.set_channel_types({x: 'eog' for x in ['EOG1', 'EOG2']})
raw.set_channel_types({x: 'seeg' for x in depth if x in raw.ch_names})
raw_an = mne.io.read_raw(fr"C:\UCLA\{subj}_clean_mtl_annot.fif")
annot = raw_an.annotations
raw = raw.set_annotations(mne.Annotations(annot.onset, annot.duration, annot.description))
raw.plot(duration=60 * 5, scalings=dict(eeg=5e-4, eog=4e-4, seeg=5e-4))

Extracting EDF parameters from C:\UCLA\P44_full_filtered_mtl.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Opening raw data file C:\UCLA\44_clean_mtl_annot.fif...


C:\Users\user\AppData\Local\Temp/ipykernel_32756/1430721516.py:9: RuntimeWarning: This filename (C:\UCLA\44_clean_mtl_annot.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_an = mne.io.read_raw(fr"C:\UCLA\{subj}_clean_mtl_annot.fif")


Isotrak not found
    Range : 0 ... 31556999 =      0.000 ... 31556.999 secs
Ready.
Opening raw data file C:\UCLA\44_clean_mtl_annot-1.fif...
Isotrak not found
    Range : 31557000 ... 40141999 =  31557.000 ... 40141.999 secs
Ready.
Using qt as 2D backend.


In [5]:
# save the fif with annotations
raw.drop_channels(raw.info['bads'], on_missing='ignore')
if subj == '025':
    # copy eog1 as 2
    raw.load_data()
    # eog2 = mne.io.RawArray(raw.copy().pick_channels(['EOG1']).get_data(),
    #                        mne.create_info(ch_names=['EOG2'], sfreq=raw.info['sfreq']))
    # raw.add_channels([eog2])
if subj == '48':
    raw.crop(tmax=60*392)
if subj == '51':
    raw.crop(tmax=60*530)
if subj == '404':
    raw.crop(tmin=93*60, tmax=60*617)
if subj == '405':
    raw.crop(tmin=909*60)
if subj == '422':
    raw.crop(tmin=57*60)
raw.save(f'C:\\UCLA\\{subj}_clean_mtl_annot.fif', overwrite=True)
# save an edf with nans
output_filename = 'clean_mtl'
nan_clean = raw.get_data(reject_by_annotation='NaN')
clean_raw = mne.io.RawArray(nan_clean, raw.info)

Reading 0 ... 32258999  =      0.000 ... 32258.999 secs...
Creating RawArray with float64 data, n_channels=1, n_times=32259000
    Range : 0 ... 32258999 =      0.000 ... 32258.999 secs
Ready.
Overwriting existing file.
Writing C:\UCLA\025_clean_mtl_annot.fif


C:\Users\user\AppData\Local\Temp/ipykernel_27748/2479625188.py:19: RuntimeWarning: This filename (C:\UCLA\025_clean_mtl_annot.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(f'C:\\UCLA\\{subj}_clean_mtl_annot.fif', overwrite=True)


Closing C:\UCLA\025_clean_mtl_annot.fif
[done]
Setting 2965945 of 32259000 (9.19%) samples to NaN, retaining 29293055 (90.81%) samples.
Creating RawArray with float64 data, n_channels=6, n_times=32259000
    Range : 0 ... 32258999 =      0.000 ... 32258.999 secs
Ready.


In [18]:
write_edf_nan(f'C:\\UCLA\\{subj}_clean_mtl.edf', clean_raw)

NameError: name 'Path' is not defined

In [7]:
subj = '018'
x = mne.io.read_raw(f'C:\\UCLA\\{subj}_clean_mtl_annot.fif')
nan_clean = x.get_data(reject_by_annotation='NaN')
clean_raw = mne.io.RawArray(nan_clean, x.info)
write_edf_nan(f'C:\\UCLA\\{subj}_clean_mtl.edf', clean_raw)

Opening raw data file C:\UCLA\018_clean_mtl_annot.fif...


C:\Users\user\AppData\Local\Temp/ipykernel_8100/2360845214.py:2: RuntimeWarning: This filename (C:\UCLA\018_clean_mtl_annot.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  x = mne.io.read_raw(f'C:\\UCLA\\{subj}_clean_mtl_annot.fif')


Isotrak not found
    Range : 0 ... 5344999 =      0.000 ...  5344.999 secs
Ready.
Setting 1582808 of 5345000 (29.61%) samples to NaN, retaining 3762192 (70.39%) samples.
Creating RawArray with float64 data, n_channels=6, n_times=5345000
    Range : 0 ... 5344999 =      0.000 ...  5344.999 secs
Ready.
